# RAG Agent with Local Gemini CLI and Conversational Memory

### A Question Answering Agent

This notebook builds and runs a complete Retrieval-Augmented Generation (RAG) agent. The agent uses a local knowledge base of PDF documents to answer questions and maintains a history of the conversation.

**Before Running:** Please ensure you have created a `.env` file with your `GOOGLE_API_KEY` and have correctly configured the path to your Gemini CLI script in the `query_with_gemini_cli` function.

In [39]:
# imports

import os
import glob
from dotenv import load_dotenv
import subprocess
import gradio as gr

In [40]:
# Load the .env file and get your API key.
# Create a file named .env and add the line:
# GOOGLE_API_KEY="your-api-key-here"
load_dotenv()
if 'GOOGLE_API_KEY' not in os.environ:
    print("ERROR: GOOGLE_API_KEY not found in environment variables.")
    print("Please create a .env file and add your key.")
    exit()

In [41]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
# Read in documents using LangChain's loaders
print("Loading documents from 'knowledge-base'...")
folders = glob.glob("knowledge-base/*")


documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

Loading documents from 'knowledge-base'...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.94s/it]


In [44]:
len(documents)

297

In [45]:
print("Splitting documents into chunks...")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Splitting documents into chunks...


In [46]:
len(chunks)

291

In [ ]:
# Create Embeddings and Store in Vector DB
print("Creating embeddings with GoogleGenerativeAIEmbeddings...")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

print("Creating FAISS vector store...")
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
print("FAISS vector store and retriever are ready.")

Creating embeddings with GoogleGenerativeAIEmbeddings...
Creating FAISS vector store...
FAISS vector store and retriever are ready.


In [ ]:
# Function to Call Gemini CLI with History
def query_with_gemini_cli(question: str, history: list):
    """
    Retrieves context, formats a prompt with history, and calls the local gemini CLI.
    """
    # Retrieve relevant documents
    print("> Retrieving context...")
    retrieved_docs = retriever.get_relevant_documents(question)
    context_string = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Format the conversation history into a string
    history_string = ""
    for user_msg, ai_msg in history:
        history_string += f"User: {user_msg}\nAI: {ai_msg}\n"

    # Build the full prompt including history, context, and the new question
    formatted_prompt = f"""Please answer the "Current Question" based on the "Context from Knowledge Base" and the "Conversation History" provided below.
Your answer should be concise and direct.

--- CONVERSATION HISTORY ---
{history_string}
--- CONTEXT FROM KNOWLEDGE BASE ---
{context_string}
--- CURRENT QUESTION ---
{question}
"""

    # Call the CLI from your local path (to update the path and run from your own machine)
    gemini_script_path = "C:\\Users\\YourUsername\\AppData\\Roaming\\npm\\gemini.ps1"
    command = [
        "powershell.exe",
        "-ExecutionPolicy", "Bypass",
        "-File", gemini_script_path,
        "-m", "gemini-2.5-flash"
    ]
    
    print(f"> Executing PowerShell command and piping prompt via stdin...")

    try:
        result = subprocess.run(
            command,
            capture_output=True,
            text=True,
            check=True,
            encoding='utf-8',
            shell=False,
            input=formatted_prompt # Pass the prompt here
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"ERROR: The CLI process returned a non-zero exit code {e.returncode}.")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}") # This will show us any errors from the script itself
        return "Sorry, the CLI process failed. Check the console for details."
    except Exception as e:
        print(f"An error occurred while calling the CLI: {e}")
        return "Sorry, I encountered an error while trying to generate a response."

In [ ]:
# Gradio Chat Function to Pass History
def chat(message, history):
    """
    Gradio chat function. It now passes the `history` to the query function.
    """
    if not message:
        return "Please ask a question."
    # Pass the current message and the history managed by Gradio
    return query_with_gemini_cli(message, history)


In [ ]:
# Launch Gradio UI
print("Launching Gradio interface...")
view = gr.ChatInterface(chat,
                        title="Expert Knowledge Worker (With Memory)",
                        description="This is a chatbot that can call an external CLI and remember conversation history.",
                       ).launch(inbrowser=True)

Launching Gradio interface...


c:\Users\YourUsername\projects\agents\.venv\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL* To create a public link, set `share=True` in `launch()`.


> Retrieving context...
> Executing PowerShell command and piping prompt via stdin...
> Retrieving context...
> Executing PowerShell command and piping prompt via stdin...
> Retrieving context...
> Executing PowerShell command and piping prompt via stdin...
> Retrieving context...
> Executing PowerShell command and piping prompt via stdin...
